In [ ]:
%matplotlib inline

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# 백엔드 설정
backend = Aer.get_backend('qasm_simulator')

# 입력 상태 구성 함수
def create_branching_circuit(input_state='0'):
    qc = QuantumCircuit(3, 3)

    # 입력 상태 준비
    if input_state == '+':
        qc.h(0)  # |+> 상태
    elif input_state == '1':
        qc.x(0)  # |1> 상태
    # 기본은 |0> 상태

    # 조건 분기 회로 구성
    qc.cx(0, 1)
    qc.ccx(0, 1, 2)

    # 측정
    qc.measure([0, 1, 2], [0, 1, 2])

    return qc

# 실험군 목록
input_conditions = ['0', '+', '1']
results = {}

# 실험 실행
for state in input_conditions:
    qc = create_branching_circuit(state)
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=1024)  # assemble 생략
    counts = job.result().get_counts()
    results[state] = counts

    # 결과 시각화
    fig = plot_histogram(counts, title=f'Input |{state}⟩')
    fig.savefig(f'branching_input_{state}.png')
    plt.close(fig)


In [ ]:
# 정량 분석 코드 삽입 :

%matplotlib inline

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine

# 정량 지표 함수 정의
def shannon_entropy(counts):
    total = sum(counts.values())
    probs = [v / total for v in counts.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def distribution_vector(counts, keys):
    total = sum(counts.values())
    return np.array([counts.get(k, 0) / total for k in keys])

# 백엔드 설정
backend = Aer.get_backend('qasm_simulator')

# 입력 상태 구성 함수
def create_branching_circuit(input_state='0'):
    qc = QuantumCircuit(3, 3)

    # 입력 상태 준비
    if input_state == '+':
        qc.h(0)  # |+> 상태
    elif input_state == '1':
        qc.x(0)  # |1> 상태

    # 조건 분기 회로 구성
    qc.cx(0, 1)
    qc.ccx(0, 1, 2)

    # 측정
    qc.measure([0, 1, 2], [0, 1, 2])

    return qc

# 실험군 목록
input_conditions = ['0', '+', '1']
results = {}

# 실험 실행
for state in input_conditions:
    qc = create_branching_circuit(state)
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=1024)
    counts = job.result().get_counts()
    results[state] = counts

    # 결과 시각화 및 저장
    fig = plot_histogram(counts, title=f'Input |{state}⟩')
    fig.savefig(f'branching_input_{state}.png')
    plt.close(fig)

# 엔트로피 계산 및 시각화
entropies = [shannon_entropy(results[state]) for state in input_conditions]

plt.bar(input_conditions, entropies)
plt.title("Shannon Entropy for Each Input")
plt.ylabel("Entropy")
plt.grid(True)
plt.savefig("entropy_comparison.png")
plt.show()

# 유사도 로그 저장
with open("similarity_log.txt", "w", encoding="utf-8") as f:
    f.write("Branching Input: Cosine Similarity Log\n------------------------\n")
    for i in range(len(input_conditions) - 1):
        s1 = input_conditions[i]
        s2 = input_conditions[i + 1]
        c1 = results[s1]
        c2 = results[s2]
        all_keys = sorted(set(c1) | set(c2))
        v1 = distribution_vector(c1, all_keys)
        v2 = distribution_vector(c2, all_keys)
        sim = 1 - cosine(v1, v2)
        f.write(f"|{s1}⟩ vs |{s2}⟩: {sim:.4f}\n")
